# E-Mail Spam Classification
## YZV 311E Term Project

Abdullah Bilici, 150200330

Bora Boyacıoğlu, 150200310

Import the necessary libraries.

In [1]:
import numpy as np
import torch
from torch import nn, optim 
from torch.nn import functional as F

from dataloader import DataLoader

## Load Data

In [2]:
# Load the data
data = np.load("../Data/bert_representations.npy")

data = torch.tensor(data)

In [3]:
# Create dataloaders so we can use it easily
train_loader = DataLoader([data[:3000, :-1].to(torch.float), data[:3000, -1]], shuffle=True, batch_size=64)
test_loader = DataLoader([data[3000:4000, :-1].to(torch.float), data[3000:4000, -1]])
validation_loader = DataLoader([data[4000:, : -1].to(torch.float), data[4000:, -1]])

## Creating a model

In [4]:
# Simple fully connected neural network
class FCNN(nn.Module):
    def __init__(self, input_shape, output_dim):
        super(FCNN, self).__init__()

        self.fc1 = nn.Linear(input_shape, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 32)
        self.fc5 = nn.Linear(32, output_dim)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)

        return x


In [5]:
N, input_shape = train_loader.shape

net = FCNN(input_shape, 2)

## Training the model

In [6]:
def test(net, test_loader, criterion, device, verbose=True):
    # We do not want to store gradients during forward pass
    with torch.no_grad():

        # Extracting the data and targets
        data, target = test_loader.X, test_loader.y
        data.to(device)
        target.to(device)
        
        # Model makes predictions
        target_pred = net(data)
        
        # Test loss
        test_loss = criterion(target_pred, target.to(torch.long)) / test_loader.size

        # Predicted values to caluclate accuracy
        target_predicted = torch.argmax(target_pred, axis = 1)

        # Calculating the accuracy
        test_accuracy = (target_predicted == target).sum() / test_loader.size
    
    if verbose:
        print("Test results: \n")

        print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}") 

    return target_predicted

In [7]:
# Setting hyperparameters
epochs = 20
lr = 2*1e-4

# Setting device to cuda if it is available
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")

# Adam for optimizer
optimizer = optim.Adam(net.parameters(), lr=lr)

# Cross entropy loss for loss function
criterion = nn.CrossEntropyLoss(reduction="sum")

Using cpu device


/home/wndpzr/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Train loop

In [8]:
train_losses = list()
validation_losses = list()

print("Training starting...")
for epoch in range(epochs):

    running_loss = .0

    for X_train, y_train in train_loader:
        X_train.to(device)
        y_train.to(device)

        # Forward pass
        y_pred = net(X_train)
        
        loss = criterion(y_pred, y_train.to(torch.long))

        # Zeros the past gradients
        optimizer.zero_grad()
        # Recalculates gradients
        loss.backward()
        # Updates weights
        optimizer.step()

        running_loss += loss.item()

    running_loss /= train_loader.size

    train_losses.append(running_loss)


    # Calculates validation loss and accuracy 
    with torch.no_grad():

        X_val, y_val = validation_loader.X, validation_loader.y

        y_val_pred = net(X_val)

        val_loss = criterion(y_val_pred, y_val.to(torch.long)) / validation_loader.size

        y_val_predicted = torch.max(y_val_pred, axis = 1).indices

        val_accuracy = (y_val_predicted == y_val).sum() / validation_loader.size

    validation_losses.append(val_loss)

    if epoch % 1 == 0:

        print(f"Epoch: {epoch+1}/{epochs},\tTraining loss: {running_loss:.4f},\tValidation loss: {val_loss:.4f},\tValidation accuracy: {val_accuracy:.4f}")
print("Training ends.")

Training starting...
Epoch: 1/20,	Training loss: 0.4251,	Validation loss: 0.2309,	Validation accuracy: 0.9334
Epoch: 2/20,	Training loss: 0.1357,	Validation loss: 0.0952,	Validation accuracy: 0.9612
Epoch: 3/20,	Training loss: 0.0771,	Validation loss: 0.0633,	Validation accuracy: 0.9757
Epoch: 4/20,	Training loss: 0.0472,	Validation loss: 0.0603,	Validation accuracy: 0.9774
Epoch: 5/20,	Training loss: 0.0499,	Validation loss: 0.0576,	Validation accuracy: 0.9780
Epoch: 6/20,	Training loss: 0.0327,	Validation loss: 0.0582,	Validation accuracy: 0.9803
Epoch: 7/20,	Training loss: 0.0320,	Validation loss: 0.1087,	Validation accuracy: 0.9606
Epoch: 8/20,	Training loss: 0.0297,	Validation loss: 0.0550,	Validation accuracy: 0.9826
Epoch: 9/20,	Training loss: 0.0200,	Validation loss: 0.0474,	Validation accuracy: 0.9815
Epoch: 10/20,	Training loss: 0.0101,	Validation loss: 0.0617,	Validation accuracy: 0.9809
Epoch: 11/20,	Training loss: 0.0097,	Validation loss: 0.0669,	Validation accuracy: 0.979

## Model Evaluation

In [9]:
val_preds = test(net, validation_loader, criterion, device, verbose = 0)
test_preds = test(net, test_loader, criterion, device)

Test results: 

Test loss: 0.0697, Test accuracy: 0.9790


In [10]:
from utils import evaluate_model

# Evaluate on validation set
print("Validation Results:")
evaluate_model(validation_loader.y, val_preds)


# Evaluate on test set
print("Test Results:")
evaluate_model(test_loader.y, test_preds)

Validation Results:
Confusion Matrix:
[[TP: 388	FP: 10	]
 [FN: 18	TN: 1312	]]

Classification Report:
Accuracy : 0.9838
Precision: 0.9749
Recall   : 0.9557
F1 Score : 0.9652

Test Results:
Confusion Matrix:
[[TP: 220	FP: 8	]
 [FN: 13	TN: 759	]]

Classification Report:
Accuracy : 0.9790
Precision: 0.9649
Recall   : 0.9442
F1 Score : 0.9544



## Save the model

In [11]:
torch.save(net, "models/FCNN_1")

## Comments

The results we got are pretty high. They are around the same as the **Random Forest** model from our default models *(from model.ipynb)*, but slightly higher.